# Investigate fast vs slow tokenizers

**WARNING**

Using a fast DeBERTa v3 tokenizer produces **unknown tokens**!

In [1]:
import os
import sys
import pandas as pd
import torch

sys.path.append('../modules/')

from model_utils import get_deberta_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

%load_ext autoreload
%autoreload 2

/home/moscato/miniconda3/envs/phb/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_DIR = '/data1/moscato/personalised-hate-boundaries-data/data/hateval2019/'

language = 'en'  # 'en' or 'es'.
partition = 'train'  # 'dev', 'test' or 'train'.

data_df = pd.read_csv(os.path.join(DATA_DIR, f'hateval2019_{language}_{partition}.csv'))[['text', 'HS']]

data_df

,text,HS
0,"Hurray, saving us $$$ in so many ways @potus @...",1
1,Why would young fighting age men be the vast m...,1
2,@KamalaHarris Illegals Dump their Kids at the ...,1
3,NY Times: 'Nearly All White' States Pose 'an A...,0
4,Orban in Brussels: European leaders are ignori...,0
...,...,...
8995,@mmdwriter @JRubinBlogger @BenSasse I am proud...,0
8996,@CheriJacobus Hollywood is complicit in the ra...,0
8997,@amaziah_filani What a fucking cunt I hate see...,1
8998,Hysterical woman like @CoryBooker,0


In [3]:
from transformers import AutoTokenizer

tokenizer_slow = AutoTokenizer.from_pretrained(
    'microsoft/deberta-v3-base',
    cache_dir='/data1/shared_models/',
    use_fast=False
)

tokenizer_fast = AutoTokenizer.from_pretrained(
    'microsoft/deberta-v3-base',
    cache_dir='/data1/shared_models/',
    use_fast=True
)

/home/moscato/miniconda3/envs/phb/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
data_df['decoded_text_slow'] = data_df['text'].map(
    lambda t: tokenizer_slow.decode(tokenizer_slow(t)['input_ids']).replace('[CLS] ', '').replace('[SEP]', '')
)

data_df['decoded_text_fast'] = data_df['text'].map(
    lambda t: tokenizer_slow.decode(tokenizer_fast(t)['input_ids']).replace('[CLS] ', '').replace('[SEP]', '')
)

In [5]:
data_df['decoded_text_fast'].str.contains('UNK').mean()

np.float64(0.009777777777777778)

In [7]:
(data_df['decoded_text_fast'] != data_df['decoded_text_slow']).mean()

np.float64(0.009666666666666667)